In [ ]:
import pandas as pd

# Load the Titanic dataset to inspect its structure and contents
file_path = /Users/aadya.mukherjee/Documents/git my/mrm_pca/titanic.csv
titanic_data = pd.read_csv(file_path)

# Display basic information about the dataset
titanic_data.info(), titanic_data.head()


SyntaxError: invalid syntax (1618818630.py, line 4)